In [1]:
from __future__ import print_function
import tensorflow as tf
import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense

from siamese import SiameseNetwork

Using TensorFlow backend.


In [2]:
dataDir = r"/home/trojan/Desktop/dimentia/data_10slices/dataset with PGGAN/train"
batch_size = 16
num_classes = 2
epochs = 100

# input image dimensions
img_height, img_width = 256, 256

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  dataDir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  dataDir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 19677 files belonging to 2 classes.
Using 15742 files for training.
Found 19677 files belonging to 2 classes.
Using 3935 files for validation.


In [4]:
class_names = train_ds.class_names
print(class_names)

['AD', 'CN']


In [5]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  print(image_batch[0].dtype)
  break

(16, 256, 256, 3)
(16,)
<dtype: 'float32'>


In [6]:
from tensorflow.keras import layers
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_ds_val = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
def create_base_model(input_shape):
    model_input = Input(shape=input_shape)

    embedding = Conv2D(32, kernel_size=(3, 3), input_shape=input_shape)(model_input)
    embedding = BatchNormalization()(embedding)
    embedding = Activation(activation='relu')(embedding)
    embedding = MaxPooling2D(pool_size=(2, 2))(embedding)
    embedding = Conv2D(64, kernel_size=(3, 3))(embedding)
    embedding = BatchNormalization()(embedding)
    embedding = Activation(activation='relu')(embedding)
    embedding = MaxPooling2D(pool_size=(2, 2))(embedding)
    embedding = Flatten()(embedding)
    embedding = Dense(128)(embedding)
    embedding = BatchNormalization()(embedding)
    embedding = Activation(activation='relu')(embedding)

    return Model(model_input, embedding)

In [9]:
def create_head_model(embedding_shape):
    embedding_a = Input(shape=128)
    embedding_b = Input(shape=128)

    head = Concatenate()([embedding_a, embedding_b])
    head = Dense(8)(head)
    head = BatchNormalization()(head)
    head = Activation(activation='sigmoid')(head)

    head = Dense(1)(head)
    head = BatchNormalization()(head)
    head = Activation(activation='sigmoid')(head)

    return Model([embedding_a, embedding_b], head)

In [10]:
def lr_schedule(epoch):

    lr = 1e-3 
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 40: # 120
        lr *= 1e-2
    elif epoch > 20: #80
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [11]:
input_shape = (256, 256, 3)
base_model = create_base_model(input_shape)
head_model = create_head_model(base_model.output_shape)

In [12]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [13]:
siamese_network = SiameseNetwork(base_model, head_model)
siamese_network.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                        optimizer=tf.keras.optimizers.Adam(lr=lr_schedule(0)),
                        metircs=[tf.keras.metrics.BinaryAccuracy()])

siamese_checkpoint_path = "./siamese_checkpoint"
filepath = siamese_checkpoint_path + '/' + 'siamese_model.h5'

Learning rate:  0.001


In [14]:
siamese_callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=10, verbose=0),
    ModelCheckpoint(filepath, monitor='val_binary_accuracy', mode='max', save_best_only=True, verbose=1),
]

In [15]:
siamese_network.fit(train_ds,
                    validation_data=val_ds,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=siamese_callbacks)

Epoch 1/100


AssertionError: in user code:

    /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/keras/eng

In [ ]:
siamese_network.load_weights(filepath)
embedding = base_model.outputs[-1]

In [ ]:
# Add softmax layer to the pre-trained embedding network
embedding = Dense(num_classes)(embedding)
embedding = BatchNormalization()(embedding)
embedding = Activation(activation='sigmoid')(embedding)

In [ ]:
model = Model(base_model.inputs[0], embedding)
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])

model_checkpoint_path = "./model_checkpoint"
filepath = model_checkpoint_path + '/' + 'best_model.h5'"

In [ ]:
model__callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=10, verbose=0),
    ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, verbose=1),
]

In [ ]:
model.fit(train_ds,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=model__callbacks,
          validation_data=val_ds)

In [ ]:
model.load_weights(model_checkpoint_path)
score = model.evaluate(val_ds, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])